In [ ]:
from sksurv.preprocessing import OneHotEncoder
from sklearn.externals import joblib
import pickle
import pandas as pd
import numpy as np

def survivalTable(modelName, raw_data):
    '''
    Calculate survival rate in years of interest
    '''
    OHE_LOCATION = "C:\\SMU_v2\\OHE\\"  
    interval = list([0.5,1,2,5,10])

    for k,v in raw_data.items():
        if str(v[0]).isalpha():
            raw_data[k] = v[0].lower()
        
    raw_data = pd.DataFrame.from_dict(raw_data)
    
    model = joblib.load('Model_folder\\{}.pkl'.format(modelName))

    with open( "{}{}{}".format(OHE_LOCATION, modelName[:-4], '_encoder.pickle'), 'rb') as f:
            enc = pickle.load(f) 
        
    #type case object to category
    typeCastList = list(raw_data.select_dtypes(include=[object]).columns)
    raw_data[typeCastList] = raw_data[typeCastList].astype("category")
    data = enc.transform(raw_data)

    surv = model.predict_survival_function(data)
    
    dic = {}
    
    for i, s in enumerate(surv):
        x = model.event_times_
        y = s
    graphaxis = pd.DataFrame({'x':x,'y':y}, columns = ['x','y'])
    for i in interval:
        result = np.where(x > (365.25*(i+1)))[0][0]
        dic[i] = y[result]

    return dic,graphaxis

# group 1
stage = "dcis/non-invasive"

if stage == "dcis/non-invasive":
    raw_data = {
            'ER': ['Positive'],\
            'PR': ['positive'],\
            'Her2': ['negative'],\
            'size_precise': [1.3],\
            'nodespos': [0],\
            'Age_@_Dx': [21],\
            'diff': ['grade 3']
           }

# group 3
elif stage == "4":
    raw_data = {
        'ER': ['Positive'],\
        'PR': ['positive'],\
        'Her2': ['negative'],\
        'size_precise': [1.3],\
        'nodespos': [0],\
        'Age_@_Dx': [21],\
        'T':['Tis'],\
        'N': ['n0'],\
        'M': ['m0']
    }

# group 2
else:
    raw_data = {
        'ER': ['Positive'],\
        'PR': ['positive'],\
        'Her2': ['negative'],\
        'Size': [">"],\
        'Age_@_Dx': [21],\
        'diff': ['m0']
    }

In [ ]:
z,DF = survivalTable('group 1_layer 4_rsf',raw_data)
DF